In [1]:
import sys
import warnings
warnings.filterwarnings("ignore")
import numpy as np
np.set_printoptions(threshold=sys.maxsize) #print out full arrays
import pandas as pd

randomstate = 42

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

import get_properties_functions_for_WI as gp

import sys

In [2]:
prefix = "pyrdz"

file_name = prefix + "_atom_map.xlsx"

atom_map_df = pd.read_excel(
    file_name, "Sheet1", index_col=0, header=0, engine="openpyxl"
)

display(atom_map_df.head())

df = atom_map_df  # df is what properties will be appended to, this creates a copy so that you have the original preserved

,log_name,C2,C3,C4,N1,N2,C1
0,pyrdz1_conf-1,C16,C17,C4,N5,N6,C7
1,pyrdz1_conf-2,C16,C17,C4,N5,N6,C7
2,pyrdz1_conf-3,C16,C17,C4,N5,N6,C7
3,pyrdz1_conf-4,C16,C17,C4,N5,N6,C7


In [3]:
# this box has functions to choose from
df = atom_map_df

# ---------------GoodVibes Engergies---------------
# uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
# calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
# inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

# ---------------Frontier Orbitals-----------------
# E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

# ---------------Polarizability--------------------
# Exact polarizability
df = gp.get_polarizability(df)

# ---------------Dipole----------------------------
# Total dipole moment magnitude in Debye
df = gp.get_dipole(df)

# ---------------Volume----------------------------
# Molar volume
# requires the Gaussian keyword = "volume" in the .com file
df = gp.get_volume(df)

# ---------------SASA------------------------------
# Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
df = gp.get_SASA(df)

# ---------------NBO-------------------------------
# natural charge from NBO
# requires the Gaussian keyword = "pop=nbo7" in the .com file
nbo_list = ["C1", "C2", "C3", "C4", "N1", "N2"]
df = gp.get_nbo(df, nbo_list)

# ---------------NMR-------------------------------
# isotropic NMR shift
# requires the Gaussian keyword = "nmr=giao" in the .com file
nmr_list = ["C1", "C2", "C3", "C4", "N1", "N2"]
df = gp.get_nmr(df, nmr_list)

# ---------------Distance--------------------------
# distance between 2 atoms
dist_list_of_lists = [["N1", "C4"], ["N2", "C1"], ["N1", "N2"]]
df = gp.get_distance(df, dist_list_of_lists)

# ---------------Angle-----------------------------
# angle between 3 atoms
angle_list_of_lists = [["C4", "N1", "N2"], ["N1", "N2", "C1"]]
df = gp.get_angles(df, angle_list_of_lists)

# ---------------Dihedral--------------------------
# dihedral angle between 4 atoms
dihedral_list_of_lists = [
    ["C3", "C4", "N1", "N2"],
    ["C2", "C1", "N2", "N2"],
    ["C4", "N1", "N2", "C1"],
]
df = gp.get_dihedral(df, dihedral_list_of_lists)

# ---------------Vbur Scan-------------------------
# uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
# inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
# if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C2", "C3", "C4", "N1", "N2"]
df = gp.get_vbur_scan(df, vbur_list, 2, 4, 0.5)

# ---------------Sterimol morfeus------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values
# NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
sterimol_list_of_lists = [["N1", "C4"], ["N2", "C1"], ["N1", "N2"]]
df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists)

# ---------------Buried Sterimol-------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
# atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
# for more information: https://kjelljorner.github.io/morfeus/sterimol.html
# inputs: dataframe, list of atom pairs, r_buried
sterimol_list_of_lists = [["N1", "C4"], ["N2", "C1"], ["N1", "N2"]]
df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5)

# ---------------Sterimol DBSTEP-------------------
# uses DBSTEP to calculate Sterimol L, B1, and B5 values
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
sterimol_list_of_lists = [["N1", "C4"], ["N2", "C1"], ["N1", "N2"]]
df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists)

# ---------------Sterimol2Vec----------------------
# uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# inputs: dataframe, list of atom pairs, end_radius, and step_size
sterimol2vec_list_of_lists = [["N1", "C4"], ["N2", "C1"], ["N1", "N2"]]
df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0)

# ---------------Pyramidalization------------------
# uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
# collects values based on two definitions of pyramidalization
# details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
pyr_list = ["C1", "C2", "C3", "C4", "N1", "N2"]
df = gp.get_pyramidalization(df, pyr_list)

# ---------------Plane Angle-----------------------
# plane angle between 2 planes (each defined by 6 atoms)
# planeangle_list_of_lists = [["N1", "C1", "C5"], ["C2", "C3", "C4"]]
# df = gp.get_planeangle(df, planeangle_list_of_lists)

# --------------LP energy - custom from first cell---------------
lp_list = ["N1", "N2"]
df = gp.get_one_lp_energy(df, lp_list)

# ---------------Time----------------------------------
# returns the total CPU time and total Wall time (not per subjob) because we are pioneers
# if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
df = gp.get_time(df)

# ---------------ChelpG----------------------------
# ChelpG ESP charge
# requires the Gaussian keyword = "pop=chelpg" in the .com file
a_list = ["C1", "C2", "C3", "C4", "N1", "N2"]
df = gp.get_chelpg(df, a_list)

# ---------------Hirshfeld-------------------------
# Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
# requires the Gaussian keyword = "pop=hirshfeld" in the .com file
a_list = ["C1", "C2", "C3", "C4", "N1", "N2"]
df = gp.get_hirshfeld(df, a_list)

pd.options.display.max_columns = None
display(df)

Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2', 'C3', 'C4', 'N1', 'N2']
NMR function has completed for ['C1', 'C2', 'C3', 'C4', 'N1', 'N2']
Distance function has completed for [['N1', 'C4'], ['N2', 'C1'], ['N1', 'N2']]
Angles function has completed for [['C4', 'N1', 'N2'], ['N1', 'N2', 'C1']]
Dihedral function has completed for [['C3', 'C4', 'N1', 'N2'], ['C2', 'C1', 'N2', 'N2'], ['C4', 'N1', 'N2', 'C1']]
Vbur scan function has completed for ['C1', 'C2', 'C3', 'C4', 'N1', 'N2'] from 2 to 4
Morfeus Sterimol function has completed for [['N1', 'C4'], ['N2', 'C1'], ['N1', 'N2']]
Morfeus Buried Sterimol function has completed for [['N1', 'C4'], ['N2', 'C1'], ['N1', 'N2']]
   pyrdz1_conf-1.log / Bmin:  2.99 / Bmax:  9.35 / L:  4.20
   pyrdz1_conf-1.log / Bmin:  2.75 / Bmax:  7.86 / L:  5.95
   pyrdz

,log_name,C2,C3,C4,N1,N2,C1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NBO_charge_C3,NBO_charge_C4,NBO_charge_N1,NBO_charge_N2,NMR_shift_C1,NMR_shift_C2,NMR_shift_C3,NMR_shift_C4,NMR_shift_N1,NMR_shift_N2,distance_N1_C4(Å),distance_N2_C1(Å),distance_N1_N2(Å),angle_C4_N1_N2(°),angle_N1_N2_C1(°),dihedral_C3_C4_N1_N2(°),dihedral_C2_C1_N2_N2(°),dihedral_C4_N1_N2_C1(°),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,%Vbur_C3_2.0Å,%Vbur_C4_2.0Å,%Vbur_N1_2.0Å,%Vbur_N2_2.0Å,%Vbur_C1_2.5Å,%Vbur_C2_2.5Å,%Vbur_C3_2.5Å,%Vbur_C4_2.5Å,%Vbur_N1_2.5Å,%Vbur_N2_2.5Å,%Vbur_C1_3.0Å,%Vbur_C2_3.0Å,%Vbur_C3_3.0Å,%Vbur_C4_3.0Å,%Vbur_N1_3.0Å,%Vbur_N2_3.0Å,%Vbur_C1_3.5Å,%Vbur_C2_3.5Å,%Vbur_C3_3.5Å,%Vbur_C4_3.5Å,%Vbur_N1_3.5Å,%Vbur_N2_3.5Å,%Vbur_C1_4.0Å,%Vbur_C2_4.0Å,%Vbur_C3_4.0Å,%Vbur_C4_4.0Å,%Vbur_N1_4.0Å,%Vbur_N2_4.0Å,Sterimol_L_N1_C4(Å)_morfeus,Sterimol_B1_N1_C4(Å)_morfeus,Sterimol_B5_N1_C4(Å)_morfeus,Sterimol_L_N2_C1(Å)_morfeus,Sterimol_B1_N2_C1(Å)_morfeus,Sterimol_B5_N2_C1(Å)_morfeus,Sterimol_L_N1_N2(Å)_morfeus,Sterimol_B1_N1_N2(Å)_morfeus,Sterimol_B5_N1_N2(Å)_morfeus,Buried_Sterimol_L_N1_C4_5.0(Å),Buried_Sterimol_B1_N1_C4_5.0(Å),Buried_Sterimol_B5_N1_C4_5.0(Å),Buried_Sterimol_L_N2_C1_5.0(Å),Buried_Sterimol_B1_N2_C1_5.0(Å),Buried_Sterimol_B5_N2_C1_5.0(Å),Buried_Sterimol_L_N1_N2_5.0(Å),Buried_Sterimol_B1_N1_N2_5.0(Å),Buried_Sterimol_B5_N1_N2_5.0(Å),Sterimol_B1_N1_C4(Å)_dbstep,Sterimol_B5_N1_C4(Å)_dbstep,Sterimol_L_N1_C4(Å)_dbstep,Sterimol_B1_N2_C1(Å)_dbstep,Sterimol_B5_N2_C1(Å)_dbstep,Sterimol_L_N2_C1(Å)_dbstep,Sterimol_B1_N1_N2(Å)_dbstep,Sterimol_B5_N1_N2(Å)_dbstep,Sterimol_L_N1_N2(Å)_dbstep,Sterimol_Bmin_N1_C4_0.0Å(Å),Sterimol_Bmax_N1_C4_0.0Å(Å),Sterimol_Bmin_N1_C4_1.0Å(Å),Sterimol_Bmax_N1_C4_1.0Å(Å),Sterimol_Bmin_N2_C1_0.0Å(Å),Sterimol_Bmax_N2_C1_0.0Å(Å),Sterimol_Bmin_N2_C1_1.0Å(Å),Sterimol_Bmax_N2_C1_1.0Å(Å),Sterimol_Bmin_N1_N2_0.0Å(Å),Sterimol_Bmax_N1_N2_0.0Å(Å),Sterimol_Bmin_N1_N2_1.0Å(Å),Sterimol_Bmax_N1_N2_1.0Å(Å),pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,pyramidalization_Gavrish_C2(°),pyramidalization_Agranat-Radhakrishnan_C2,pyramidalization_Gavrish_C3(°),pyramidalization_Agranat-Radhakrishnan_C3,pyramidalization_Gavrish_C4(°),pyramidalization_Agranat-Radhakrishnan_C4,pyramidalization_Gavrish_N1(°),pyramidalization_Agranat-Radhakrishnan_N1,pyramidalization_Gavrish_N2(°),pyramidalization_Agranat-Radhakrishnan_N2,NBO_LP_occupancy_N1,NBO_LP_energy_N1,NBO_LP_occupancy_N2,NBO_LP_energy_N2,CPU_time_total(hours),Wall_time_total(hours),ChelpG_charge_C1,ChelpG_charge_C2,ChelpG_charge_C3,ChelpG_charge_C4,ChelpG_charge_N1,ChelpG_charge_N2,Hirsh_charge_C1,Hirsh_CM5_charge_C1,Hirsh_atom_dipole_C1,Hirsh_charge_C2,Hirsh_CM5_charge_C2,Hirsh_atom_dipole_C2,Hirsh_charge_C3,Hirsh_CM5_charge_C3,Hirsh_atom_dipole_C3,Hirsh_charge_C4,Hirsh_CM5_charge_C4,Hirsh_atom_dipole_C4,Hirsh_charge_N1,Hirsh_CM5_charge_N1,Hirsh_atom_dipole_N1,Hirsh_charge_N2,Hirsh_CM5_charge_N2,Hirsh_atom_dipole_N2
0,pyrdz1_conf-1,C16,C17,C4,N5,N6,C7,-723.266926,0.223195,-723.027121,0.062668,0.059724,-723.089789,-723.086846,298.15,-0.28789,-0.03793,-0.16291,0.24996,0.05309,200.979,205.946,2.7605,2558.739,467.943546,722.480854,0.832099,0.13315,-0.18589,-0.20864,0.18810,-0.21950,-0.20526,5.5557,39.1191,41.6092,-11.9697,-283.3194,-282.4306,1.34220,1.34655,1.32316,120.578,121.007,0.630,NaN,0.265,95.751550,92.697572,92.416710,96.797521,80.878745,81.714876,86.734926,79.251856,77.427074,89.930329,72.068303,71.129053,76.032665,66.550581,64.177034,80.180088,63.974970,61.410533,64.506266,56.051994,54.637416,68.196166,57.357086,54.066693,53.587611,47.678217,47.185541,55.247191,50.853842,47.966064,4.653025,3.007906,9.350620,6.399445,2.724532,7.895892,10.772926,2.199199,4.765490,4.653025,2.925634,6.562661,5.987262,2.545027,5.763776,7.519437,2.058946,4.765490,2.988738,9.

## Save collected properties to Excel

Helpful to save here in case the Notebook crashes or if you want to add more properties before post-processsing. Can be read in at 5.1.1.

In [4]:
writer = pd.ExcelWriter(prefix + "_extracted_properties.xlsx", engine="xlsxwriter")
df.to_excel(writer)
writer.close()

In [5]:
# save the pandas dataframe to a pickle file
df.to_pickle(prefix + "_extracted_properties.pkl")